In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as pl
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object
df = pd.DataFrame.from_records(shelter.readAll({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Logo ADDED
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Carlie'))),
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='radio-button',
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain', 'value': 'mountain'},
                {'label': 'Disaster', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='Water'
        ),
        html.Div(id='dd-output-container')
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),

    
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 dcc.Graph(
                     id="graph-id",
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
               columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
               data=df.to_dict('records')
               return (data,columns)

@app.callback(
    Output(component_id='datatable-id', component_property= 'data'),
    [dash.dependencies.Input('radio-button', 'value')])
def update_output(value):
    if (value == 'mountain'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value =='water'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
    else:
        df = pd.DataFrame.from_records(shelter.readAll({}))
    return df.to_dict('records')

    


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output("graph-id", "figure"), 
    Input("names", "value"), 
    Input("values", "value"))
def generate_chart(names, values):
    dff =pd.DataFrame.from_records(shelter.readAll({}))
    fig = px.pie(dff, values=values, names=names, hole=.3)
    return fig




@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData):
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[latitude, longitude], children=[
                #dl.Tooltip(dff.iloc[0, 4]),
                #dl.Tooltip(viewData.iloc[0, 4]),
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    #html.P(dff.iloc[1, 9])
                    html.P(name)
                ])
            ])
        ])
    ]

app